# 1 读取xls内部1984-2021年数据

### 提示：xls内部含有多个sheet表，表的前两行均为列名且含有合并的单元格
###       这里首先读取并按照其格式获取数据，并保存新数据到csv文件内

In [1]:
import pandas as pd  
  
# 读取xls文件  
xls_file = pd.ExcelFile('1.碧流河水库1984-2021年雨水情信息统计表.xls')  

#在这里修改需要获取的数据列名即可
_columns_ = '西扒山'  
  
# 获取所有工作表的名称  
sheet_names = xls_file.sheet_names  
  
# 初始化一个空的DataFrame来存储所有数据  
all_data = pd.DataFrame()  
  
# 遍历每个工作表  
for sheet_name in sheet_names:  
    # 读取工作表数据  
    sheet_data = pd.read_excel(xls_file, sheet_name, header=1)  
  
    # 提取需要的一列数据（假设列名为"降雨量"）  
    column_data = sheet_data[_columns_]  
  
    # 检查数据长度是否匹配年份（考虑闰年）  
    if len(column_data) == 365:  
        # 如果不是闰年，直接添加数据到总的DataFrame中  
        all_data[sheet_name] = column_data.tolist()  
    elif len(column_data) == 366:  
        # 如果是闰年，添加数据到总的DataFrame中，并忽略最后一天的数据  
        all_data[sheet_name] = column_data[:-1].tolist()  
    else:  
        print(f"Warning: Unexpected length of data for {sheet_name}: {len(column_data)}")  
        # 如果数据长度既不是365也不是366，根据具体情况处理  
        # 删除第59行数据 即2月29日，非汛期且只有一天 
        column_data = column_data.drop(59)  # 注意索引是从0开始的，所以第59行的索引是58 
        # 这里只是简单地忽略了最后一天的数据  
        all_data[sheet_name] = column_data[:-1].tolist()  
  
# 显示结果  
print(all_data)
#all_data = all_data.drop(all_data.columns[0], axis=1)
#存储数据
csv_name = '碧流河抽取{}站1984-2021.csv'.format(_columns_)  
all_data.to_csv(csv_name)

     1984  1985  1986  1987  1988  1989  1990  1991  1992  1993  ...  2012  \
0     0.0   0.0   0.0   0.0   0.0   1.1   0.0   0.0   0.0   0.0  ...   0.0   
1     0.0   0.1   4.1   0.0   0.0   0.0   0.0   1.9   0.0   0.0  ...   0.7   
2     0.0   0.0   0.5   0.0   0.5   0.0   0.0   0.0   0.0   0.0  ...   0.1   
3     0.0   0.0   0.0   1.2   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4     0.0   0.0   0.0   1.6   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
..    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
360   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
361   0.0   1.5   0.0   0.0   0.0   0.4   0.0   0.0   0.0   0.0  ...   7.4   
362   0.0   0.5   0.4   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   1.1   
363   0.0   0.0   0.8   0.0   0.0   0.0   3.2   0.0   0.0   1.8  ...   0.0   
364   0.0   0.0   0.0   0.5   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.8   

     2013  2014  2015  2016  2017  2018  2019  2020  2021  
0  

## 提示：上面获取的是按照年份排列的数据，因为格式要统一所有的闰年都去掉了2月29日（非汛期的一天影响不大）
## 因此下面的数据查看csv文件会发现最后一行并不是2021年12月30日，相差的几天就是这个问题，但是对于分析影响不大

In [2]:
import pandas as pd  
  
# 读取CSV文件  
df = pd.read_csv(csv_name)  
# 删除第一列（索引）  
df = df.drop(columns=[df.columns[0]])     
print(df)
# 把数据都拼在一起变成一列
merged_array = df.values.flatten()

print(len(merged_array))
import pandas as pd  
import numpy as np  
from datetime import datetime , timedelta  
  
# 假设你已经有一个合并的数组 `merged_array`  
# 创建一个空的DataFrame  
df = pd.DataFrame(columns=['Time', 'Value'])  
  
# 添加时间索引列  
for i in range(len(merged_array)):  
    # 生成日期  
    date = datetime(1984, 1, 1) + timedelta(days=i) 
       # 检查日期是否是2月29日  
    if date.month != 2 or date.day != 29:   
    # 将日期转换为字符串格式，并将其添加到DataFrame中  
        df = df.append({'Time': date.strftime('%Y-%m-%d'), 'Value': merged_array[i]}, ignore_index=True)  
#df = df[df['Value'] > '0']  
print(df)
csv_name = '碧流河抽取{}站合并1984-2021.csv'.format(_columns_)  
df.to_csv(csv_name)

     1984  1985  1986  1987  1988  1989  1990  1991  1992  1993  ...  2012  \
0     0.0   0.0   0.0   0.0   0.0   1.1   0.0   0.0   0.0   0.0  ...   0.0   
1     0.0   0.1   4.1   0.0   0.0   0.0   0.0   1.9   0.0   0.0  ...   0.7   
2     0.0   0.0   0.5   0.0   0.5   0.0   0.0   0.0   0.0   0.0  ...   0.1   
3     0.0   0.0   0.0   1.2   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4     0.0   0.0   0.0   1.6   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
..    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
360   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
361   0.0   1.5   0.0   0.0   0.0   0.4   0.0   0.0   0.0   0.0  ...   7.4   
362   0.0   0.5   0.4   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   1.1   
363   0.0   0.0   0.8   0.0   0.0   0.0   3.2   0.0   0.0   1.8  ...   0.0   
364   0.0   0.0   0.0   0.5   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.8   

     2013  2014  2015  2016  2017  2018  2019  2020  2021  
0  

/tmp/ipykernel_2386836/3289559811.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Time': date.strftime('%Y-%m-%d'), 'Value': merged_array[i]}, ignore_index=True)
/tmp/ipykernel_2386836/3289559811.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Time': date.strftime('%Y-%m-%d'), 'Value': merged_array[i]}, ignore_index=True)
/tmp/ipykernel_2386836/3289559811.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Time': date.strftime('%Y-%m-%d'), 'Value': merged_array[i]}, ignore_index=True)
/tmp/ipykernel_2386836/3289559811.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = 

             Time Value
0      1984-01-01   0.0
1      1984-01-02   0.0
2      1984-01-03   0.0
3      1984-01-04   0.0
4      1984-01-05   0.0
...           ...   ...
13855  2021-12-17   0.0
13856  2021-12-18   0.0
13857  2021-12-19   0.0
13858  2021-12-20   0.0
13859  2021-12-21   0.0

[13860 rows x 2 columns]


/tmp/ipykernel_2386836/3289559811.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Time': date.strftime('%Y-%m-%d'), 'Value': merged_array[i]}, ignore_index=True)
/tmp/ipykernel_2386836/3289559811.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Time': date.strftime('%Y-%m-%d'), 'Value': merged_array[i]}, ignore_index=True)
/tmp/ipykernel_2386836/3289559811.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Time': date.strftime('%Y-%m-%d'), 'Value': merged_array[i]}, ignore_index=True)
/tmp/ipykernel_2386836/3289559811.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = 